# Python для анализа данных

## Использование API. Работа с форматами XML и JSON

*На основе лекции Ильи Щурова, НИУ ВШЭ*  
*Дополнения: Татьяна Рогович, НИУ ВШЭ*  


# XML

До этого мы с вами собирали данные вручную, обращаясь к html страницам, размеченным для отображения в браузере. Но данные также можно собирать и через API -  — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером. 

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

```xml
<group>
    <number>134</number>
    <student>
        <firstname>Виталий</firstname>
        <lastname>Иванов</lastname>
    </student>
    <student>
        <firstname>Мария</firstname>
        <lastname>Петрова</lastname>
    </student>
</group>
```

Для обработки XML-файлов можно использовать тот же пакет *Beautiful Soup*, который мы уже использовали для работы с HTML. Единственное различие — нужно указать дополнительный параметр `feautres="xml"` при вызове функции `BeautifulSoup` — чтобы он не искал в документе HTML-теги.

In [4]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [2]:
!pip install lxml

In [6]:
from bs4 import BeautifulSoup

obj = BeautifulSoup(group, "lxml")
print(obj.prettify())

<html>
 <body>
  <group>
   <number>
    134
   </number>
   <student>
    <firstname>
     Виталий
    </firstname>
    <lastname>
     Иванов
    </lastname>
   </student>
   <student>
    <firstname>
     Мария
    </firstname>
    <lastname>
     Петрова
    </lastname>
   </student>
  </group>
 </body>
</html>


Номер группы можно найти, например, вот так - для каждого объекта через точку указываем его атрибут, в который надо спуститься. 

In [4]:
obj.group.number.text # последний атрибут текст, точно также как делали в html

'134'

Но это работает только тогда, когда тэг уникальный. В других случаях, парсер всегда будет попадать в первый child-тэг, который он встретил по пути вниз.

In [5]:
obj.group.student.lastname.text # до Петровой так не добраться

'Иванов'

Перечислить всех студентов можно с помощью цикла (похожая структура у нас была и в обработке html).

In [12]:
for student in obj.group.find_all('student'):
    print(student.lastname.text, student.firstname.text)

Иванов Виталий
Петрова Мария


По сути, главное отличие xml от html, что работать вы будете не со стандартизированными структурами. Поэтому перед работой придется поиграть в детективов - запросить данные и внимательно изучить расположение узлов, чтобы понять, какие тэги вас интересуют.

XML легко представить в виде дерева, где есть главный узел (parent) и его "дети".

![](https://www.py4e.com/images/xml-tree.svg)  
*Источник: Python for Everybody, C.Severance*

Кроме BS парсить xml можно и с помощью других библиотек. Например, ElementTree.


In [8]:
import xml.etree.ElementTree as ET

In [10]:
tree = ET.fromstring(group)
list(tree) # посмотрим, что внутри

[<Element 'number' at 0x7ff1a155d9f0>,
 <Element 'student' at 0x7ff1a155d4f0>,
 <Element 'student' at 0x7ff1a155d450>]

Cинтаксис очень похож на BS. Добрались до первой фамилии.

In [9]:
tree.find('student').find('lastname').text

'Иванов'

In [13]:
for element in tree.findall('student'):
    print(element)
    print(element.find('lastname').text)

<Element 'student' at 0x7ff1a155d4f0>
Иванов
<Element 'student' at 0x7ff1a155d450>
Петрова


Можно немного упростить код, включив дочерний тэг в findall.

In [12]:
for element in tree.findall('student/lastname'):
    print(element.text)

Иванов
Петрова


# Задача
По ссылке данные в формате xml.
http://py4e-data.dr-chuck.net/comments_42.xml

Посчитайте все комментарии в этом документе (поля count).

In [14]:
import requests
data = requests.get('http://py4e-data.dr-chuck.net/comments_42.xml').text

tree = ET.fromstring(data)

total = 0
for element in tree.findall('comments/comment/count'):
#     print(type(element.text)
    total += int(element.text)
    
print(total)

2553


In [15]:
sum([int(element.text) for element in tree.findall('comments/comment/count')])

2553

# Реальный пример: wiki API

Допустим, нам потребовалось получить список всех статей из некоторой категории в Википедии. Мы могли бы открыть эту категорию в браузере и дальше действовать теми методами, которые обсуждались выше. Однако, на наше счастье разработчики Википедии сделали удобное API. Чтобы научиться с ним работать, придётся познакомиться с [документацией](https://www.mediawiki.org/wiki/API:Main_page) (так будет с любым API), но это кажется сложным только в первый раз. Ну хорошо, в первые 10 раз. Или 20. Потом будет проще.

Многие API будут требовать токена (например, ваш google логин-пароль для работы с гугл-документами), но мы сейчас работаем с открытым интерфейсом.

Итак, приступим. Взаимодействие с сервером при помощи API происходит с помощью отправки специальным образом сформированных запросов и получения ответа в одном из машинночитаемых форматов. Нас будет интересовать формат XML, хотя бывают и другие (позже мы познакомимся с JSON). А вот такой запрос мы можем отправить:

https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:Physics&cmsort=timestamp&cmdir=desc&format=xmlfm

Строка `https://en.wikipedia.org/w/api.php` (до знака вопроса) — это *точка входа* в API. Всё, что идёт после знака вопроса — это, собственно, запрос. Он представляет собой что-то вроде словаря и состоит из пар «ключ=значение», разделяемых амперсандом `&`. Некоторые символы приходится кодировать специальным образом.

Например, в адресе выше сказано, что мы хотим сделать запрос (`action=query`), перечислить элементы категории `list=categorymembers`, в качестве категории, которая нас интересует, указана `Category:Physics` (`cmtitle=Category:Physics`) и указаны некоторые другие параметры. Если кликнуть по этой ссылке, откроется примерно такая штука:

```xml
<?xml version="1.0"?>
<api batchcomplete="">
  <continue cmcontinue="2015-05-30 19:37:50|1653925" continue="-||" />
  <query>
    <categorymembers>
      <cm pageid="24293838" ns="0" title="Wigner rotation" />
      <cm pageid="48583145" ns="0" title="Northwest Nuclear Consortium" />
      <cm pageid="48407923" ns="0" title="Hume Feldman" />
      <cm pageid="48249441" ns="0" title="Phase Stretch Transform" />
      <cm pageid="47723069" ns="0" title="Epicatalysis" />
      <cm pageid="2237966" ns="14" title="Category:Surface science" />
      <cm pageid="2143601" ns="14" title="Category:Interaction" />
      <cm pageid="10844347" ns="14" title="Category:Physical systems" />
      <cm pageid="18726608" ns="14" title="Category:Physical quantities" />
      <cm pageid="22688097" ns="0" title="Branches of physics" />
    </categorymembers>
  </query>
</api>
```

Мы видим здесь разные теги, и видим, что нас интересуют теги `<cm>`, находящиеся внутри тега `<categorymembers>`.

Давайте сделаем соответствующий запрос с помощью Python. Для этого нам понадобится уже знакомый модуль `requests`.

In [16]:
import requests
url = "https://en.wikipedia.org/w/api.php"
params = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'xml'
}

g = requests.get(url, params=params)

Как видно, список параметров мы передаем в виде обычного словаря. Посмотрим, что получилось.

In [18]:
g.text

'<?xml version="1.0"?><api batchcomplete=""><continue cmcontinue="subcat|403a4e504e030648385a4e3a2e4e050e4c32402a50323004403a4e504e011f01dcc1dc18|37358141" continue="-||" /><query><categorymembers><cm pageid="22939" ns="0" title="Physics" /><cm pageid="1653925" ns="100" title="Portal:Physics" /><cm pageid="23479" ns="0" title="Physicalism" /><cm pageid="70983414" ns="14" title="Category:Physics by country" /><cm pageid="49740128" ns="14" title="Category:Subfields of physics" /><cm pageid="694942" ns="14" title="Category:Physicists" /><cm pageid="5625591" ns="14" title="Category:Physics awards" /><cm pageid="36477012" ns="14" title="Category:Concepts in physics" /><cm pageid="71976587" ns="14" title="Category:Physics events" /><cm pageid="1310583" ns="14" title="Category:History of physics" /></categorymembers></query></api>'

In [51]:
?g.ok # возвращает ошибку, если сервер или клиент не отвечает

Type:        property
String form: <property object at 0x00000223E404E408>
Docstring:  
Returns True if :attr:`status_code` is less than 400.

This attribute checks if the status code of the response is between
400 and 600 to see if there was a client error or a server error. If
the status code, is between 200 and 400, this will return True. This
is **not** a check to see if the response code is ``200 OK``.


Всё хорошо. Теперь используем *Beautiful Soup* для обработки этого XML.

In [19]:
data = BeautifulSoup(g.text, features='lxml')

In [20]:
print(data.prettify())

<?xml version="1.0"?>
<html>
 <body>
  <api batchcomplete="">
   <continue cmcontinue="subcat|403a4e504e030648385a4e3a2e4e050e4c32402a50323004403a4e504e011f01dcc1dc18|37358141" continue="-||">
   </continue>
   <query>
    <categorymembers>
     <cm ns="0" pageid="22939" title="Physics">
     </cm>
     <cm ns="100" pageid="1653925" title="Portal:Physics">
     </cm>
     <cm ns="0" pageid="23479" title="Physicalism">
     </cm>
     <cm ns="14" pageid="70983414" title="Category:Physics by country">
     </cm>
     <cm ns="14" pageid="49740128" title="Category:Subfields of physics">
     </cm>
     <cm ns="14" pageid="694942" title="Category:Physicists">
     </cm>
     <cm ns="14" pageid="5625591" title="Category:Physics awards">
     </cm>
     <cm ns="14" pageid="36477012" title="Category:Concepts in physics">
     </cm>
     <cm ns="14" pageid="71976587" title="Category:Physics events">
     </cm>
     <cm ns="14" pageid="1310583" title="Category:History of physics">
     </cm>
   

Найдём все вхождения тега `<cm>` и выведем их атрибут `title`:

In [21]:
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Physics
Portal:Physics
Physicalism
Category:Physics by country
Category:Subfields of physics
Category:Physicists
Category:Physics awards
Category:Concepts in physics
Category:Physics events
Category:History of physics


Можно было упростить поиск `<cm>`, не указывая «полный путь» к ним:

In [22]:
for cm in data("cm"):
    print(cm['title'])

Physics
Portal:Physics
Physicalism
Category:Physics by country
Category:Subfields of physics
Category:Physicists
Category:Physics awards
Category:Concepts in physics
Category:Physics events
Category:History of physics


По умолчанию сервер вернул нам список из 10 элементов. Если мы хотим больше, нужно воспользоваться элементом `continue` — это своего рода гиперссылка на следующие 10 элементов.

In [23]:
data.find("continue")['cmcontinue']

'subcat|403a4e504e030648385a4e3a2e4e050e4c32402a50323004403a4e504e011f01dcc1dc18|37358141'

Мне пришлось использовать метод `find()` вместо того, чтобы просто написать `data.continue`, потому что `continue` в Python имеет специальный смысл.

Теперь добавим `cmcontinue` в наш запрос и выполним его ещё раз:

In [24]:
params['cmcontinue'] = data.api("continue")[0]['cmcontinue']

In [25]:
params

{'action': 'query',
 'list': 'categorymembers',
 'cmtitle': 'Category:Physics',
 'format': 'xml',
 'cmcontinue': 'subcat|403a4e504e030648385a4e3a2e4e050e4c32402a50323004403a4e504e011f01dcc1dc18|37358141'}

In [26]:
g = requests.get(url, params=params)
data = BeautifulSoup(g.text, features='xml')
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Category:Physics-related lists
Category:Physics literature
Category:Physical modeling
Category:Physics organizations
Category:Physical systems
Category:Works about physics
Category:Physics stubs


Мы получили следующие 10 элементов из категории. Продолжая таким образом, можно выкачать её даже целиком (правда, для этого потребуется много времени).

Аналогичным образом реализована работа с разнообразными другими API, имеющимися на разных сайтах. Где-то API является полностью открытым (как в Википедии), где-то вам потребуется зарегистрироваться и получить application id и какой-нибудь ключ для доступа к API, где-то попросят даже заплатить (например, автоматический поиск в Google стоит что-то вроде 5 долларов за 100 запросов). Есть API, которые позволяют только читать информацию, а бывают и такие, которые позволяют её править. Например, можно написать скрипт, который будет автоматически сохранять какую-то информацию в Google Spreadsheets. Всякий раз при использовании API вам придётся изучить его документацию, но это в любом случае проще, чем обрабатывать HTML-код. Иногда удаётся упростить доступ к API, используя специальные библиотеки.

# JSON

Другой популярный формат, в котором клиент может отдать вам данные - json. JSON расшифровывается как JavaScript Object Notation и изначально возник как подмножество языка JavaScript (пусть вас не вводит в заблуждение название, этот язык ничего не имеет общего с Java), используемое для описания объектов, но впоследствии стал использоваться и в других языках программирования, включая Python. Различные API могут поддерживать либо XML, либо JSON, либо и то, и другое, так что нам полезно научиться работать с обоими типами данных (например, wiki api могла бы выгрузить нам данные и в формате json при соответствующем запросе).

In [27]:
url = "https://en.wikipedia.org/w/api.php"
params = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'json' # поменяли формат на json
}

j = requests.get(url, params=params)
j.ok

True

In [28]:
j.text

'{"batchcomplete":"","continue":{"cmcontinue":"subcat|403a4e504e030648385a4e3a2e4e050e4c32402a50323004403a4e504e011f01dcc1dc18|37358141","continue":"-||"},"query":{"categorymembers":[{"pageid":22939,"ns":0,"title":"Physics"},{"pageid":1653925,"ns":100,"title":"Portal:Physics"},{"pageid":23479,"ns":0,"title":"Physicalism"},{"pageid":70983414,"ns":14,"title":"Category:Physics by country"},{"pageid":49740128,"ns":14,"title":"Category:Subfields of physics"},{"pageid":694942,"ns":14,"title":"Category:Physicists"},{"pageid":5625591,"ns":14,"title":"Category:Physics awards"},{"pageid":36477012,"ns":14,"title":"Category:Concepts in physics"},{"pageid":71976587,"ns":14,"title":"Category:Physics events"},{"pageid":1310583,"ns":14,"title":"Category:History of physics"}]}}'

Посмотрим, что достали. Уже сразу видно, что структура у данных совсем другая.

JSON очень похож на описание объекта в Python и смысл квадратных и фигурных скобок такой же. Правда, есть и отличия: например, в Python одинарные и двойные кавычки ничем не отличаются, а в JSON можно использовать только двойные. Мы видим, что полученный нами JSON представляет собой словарь, значения которого — строки или числа, а также списки или словари, значения которых в свою очередь также могут быть строками, числами, списками, словарями и т.д. То есть получается такая довольно сложная структура данных.

В данный момент тот факт, что перед нами сложная структура данных, видим только мы — с точки зрения Python, j.text это просто такая строка. Однако в модуле requests есть метод, позволяющий сразу выдать питоновский объект (словарь или список), если результат запроса возвращён в формате JSON. Так что нам не придётся использовать никакие дополнительные библиотеки.

In [29]:
j_data = j.json()

In [32]:
j_data # получили честный питоновский словарь

{'batchcomplete': '',
 'continue': {'cmcontinue': 'subcat|403a4e504e030648385a4e3a2e4e050e4c32402a50323004403a4e504e011f01dcc1dc18|37358141',
  'continue': '-||'},
 'query': {'categorymembers': [{'pageid': 22939, 'ns': 0, 'title': 'Physics'},
   {'pageid': 1653925, 'ns': 100, 'title': 'Portal:Physics'},
   {'pageid': 23479, 'ns': 0, 'title': 'Physicalism'},
   {'pageid': 70983414, 'ns': 14, 'title': 'Category:Physics by country'},
   {'pageid': 49740128, 'ns': 14, 'title': 'Category:Subfields of physics'},
   {'pageid': 694942, 'ns': 14, 'title': 'Category:Physicists'},
   {'pageid': 5625591, 'ns': 14, 'title': 'Category:Physics awards'},
   {'pageid': 36477012, 'ns': 14, 'title': 'Category:Concepts in physics'},
   {'pageid': 71976587, 'ns': 14, 'title': 'Category:Physics events'},
   {'pageid': 1310583, 'ns': 14, 'title': 'Category:History of physics'}]}}

Содержательная информация хранится по ключу 'query'. А уже внутри есть ключ 'categorymembers', значением которого является список всех категорий. Каждая категория отображается в виде словаря, записями которого являются разные параметры категории (например, 'title' соответствует названию, а pageid — внутреннему идентификатору в системе).



In [33]:
j_data['query']['categorymembers'] # привычный нам список

[{'pageid': 22939, 'ns': 0, 'title': 'Physics'},
 {'pageid': 1653925, 'ns': 100, 'title': 'Portal:Physics'},
 {'pageid': 23479, 'ns': 0, 'title': 'Physicalism'},
 {'pageid': 70983414, 'ns': 14, 'title': 'Category:Physics by country'},
 {'pageid': 49740128, 'ns': 14, 'title': 'Category:Subfields of physics'},
 {'pageid': 694942, 'ns': 14, 'title': 'Category:Physicists'},
 {'pageid': 5625591, 'ns': 14, 'title': 'Category:Physics awards'},
 {'pageid': 36477012, 'ns': 14, 'title': 'Category:Concepts in physics'},
 {'pageid': 71976587, 'ns': 14, 'title': 'Category:Physics events'},
 {'pageid': 1310583, 'ns': 14, 'title': 'Category:History of physics'}]

In [34]:
for cm in j_data['query']['categorymembers']: # пройдемся по нему привычным нам циклом
    print(cm['title'])

Physics
Portal:Physics
Physicalism
Category:Physics by country
Category:Subfields of physics
Category:Physicists
Category:Physics awards
Category:Concepts in physics
Category:Physics events
Category:History of physics


Преимущества JSON в том, что мы получаем готовый объект Python и нет необходимости использовать какие-то дополнительные библиотеки для того, чтобы с ним работать. Недостатком является то же самое: зачастую поиск информации в XML-файле может проводиться более эффективно, чем в JSON. Продемонстрируем это на уже рассмотренном примере. Чтобы получить список всех тегов <cm>, в которых хранилась информация об элементах категории в XML, мы использовали полный «путь»:

```python
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])
```

Однако, это можно бы сделать (в данном случае) гораздо короче. Если посмотреть на XML, то можно заметить, что в нём нет других тегов <cm>, кроме тех, которые нам нужны. С другой стороны, Beautiful Soup ищет все теги с данным именем, а не только те, которые являются потомками первого уровня для данного тега. Таким образом, код выше можно было бы переписать более коротко:

In [35]:
for cm in data("cm"):
    print(cm['title'])

Category:Physics-related lists
Category:Physics literature
Category:Physics organizations
Category:Physics stubs


Конечно data("cm") выглядит короче, чем q['query']['categorymembers']. В JSON мы не можем использовать подобные методы. Так что у обоих форматов есть свои плюсы и минусы.

## JSON (парсинг VK)

Как уже говорилось выше, не все API открытые. Так, чтобы достать информацию из vk вам придется сгенерировать токен с помощью вашего аккаунта (мы выложим отдельный блокнот как это сделать, для тех, кому будет интересно). API VK отдает данные в json. Структура тут будет посложнее, чем то, что мы уже видели, поэтому давайте еще потренируемся.

Теперь научимся еще и загружать JSON файл с диска. Для этого нам понадобится модуль json.

In [37]:
import json
with open('vk.json',  'r', encoding='Utf-8') as json_data:
    res_loaded = json.load(json_data) # считываем данные с помощью функции .load()

Здесь у нас выгрузка постов со стены группы ВШЭ.

In [38]:
res_loaded

{'count': 1542,
 'items': [{'id': 32494,
   'from_id': -132,
   'owner_id': -132,
   'date': 1541422799,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': '',
   'attachments': [{'type': 'link',
     'link': {'url': 'http://family.hse.ru/event/view/2271',
      'title': 'День карьеры факультета бизнеса и менеджмента',
      'caption': 'family.hse.ru',
      'description': '',
      'photo': {'id': 456239093,
       'album_id': -2,
       'owner_id': 100,
       'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
       'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
       'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
       'width': 150,
       'height': 80,
       'text': '',
       'date': 1541422799}}}],
   'post_source': {'type': 'vk'},
   'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
   'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
  

In [40]:
res_loaded['items'][:2]

[{'id': 32494,
  'from_id': -132,
  'owner_id': -132,
  'date': 1541422799,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': '',
  'attachments': [{'type': 'link',
    'link': {'url': 'http://family.hse.ru/event/view/2271',
     'title': 'День карьеры факультета бизнеса и менеджмента',
     'caption': 'family.hse.ru',
     'description': '',
     'photo': {'id': 456239093,
      'album_id': -2,
      'owner_id': 100,
      'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
      'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
      'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
      'width': 150,
      'height': 80,
      'text': '',
      'date': 1541422799}}}],
  'post_source': {'type': 'vk'},
  'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
  'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
  'reposts': {'count': 2, 'user_reposted': 0},
  'vi

In [41]:
res_loaded.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [42]:
res_loaded['items'][0] # первый элемент items - первый пост со всей информацией о нем

{'id': 32494,
 'from_id': -132,
 'owner_id': -132,
 'date': 1541422799,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'attachments': [{'type': 'link',
   'link': {'url': 'http://family.hse.ru/event/view/2271',
    'title': 'День карьеры факультета бизнеса и менеджмента',
    'caption': 'family.hse.ru',
    'description': '',
    'photo': {'id': 456239093,
     'album_id': -2,
     'owner_id': 100,
     'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
     'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
     'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
     'width': 150,
     'height': 80,
     'text': '',
     'date': 1541422799}}}],
 'post_source': {'type': 'vk'},
 'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
 'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
 'reposts': {'count': 2, 'user_reposted': 0},
 'views': {'count': 2832}}

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста и дате публикации. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [43]:
items = res_loaded['items']
full_list = []

for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']] # нужные поля
    full_list.append(l) # добавляем в список списков full_list
# несколько элементов списка
full_list[0:4]

[[-132, 32494, '', 1541422799],
 [-132,
  32493,
  'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
  1541076555],
 [-132, 32492, '', 1540743239],
 [-132, 32491, '', 1540551399]]

Видно, что в двух первых постах текста не обнаружено, там только картинки, ссылки и репосты.

Из этого списка списков можно легко сделать датафрейм `pandas`.

Но если у ссылок, репостов и картинок нет текста, то наш DataFrame будет выглядет неполноценно. Поэтому добавим условие, что если текст отсутсвует, вставим в DataFrame строку с этой отметкой.

In [44]:
# опять выберем только нужные поля
full_list = []
for item in items:
    if item['text'] == "":
        l = [item['from_id'], item['id'], "Картинка, ссылка или репост", item['date']]
    else:
        l = [item['from_id'], item['id'], item['text'], item['date']]
    full_list.append(l)

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [45]:
import pandas as pd

Создадим датафрейм:

In [46]:
df = pd.DataFrame(full_list)
df.head(10)

,0,1,2,3
0,-132,32494,"Картинка, ссылка или репост",1541422799
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555
2,-132,32492,"Картинка, ссылка или репост",1540743239
3,-132,32491,"Картинка, ссылка или репост",1540551399
4,-132,32488,"Картинка, ссылка или репост",1540302306
5,-132,32486,26-28 октября в Москве пройдет крупнейший в Ро...,1539863523
6,-132,32485,"Картинка, ссылка или репост",1539781575
7,-132,32484,"Картинка, ссылка или репост",1539718883
8,-132,32481,"Картинка, ссылка или репост",1539088885
9,-132,32480,Будущим магистрам! Новая программа Факультета ...,1539088745


Ура! Осталось только дать внятные названия столбцам и разобраться, почему дата представлена в таком виде. что делать со столбцами, мы уже знаем.

In [47]:
df.columns = ['From_id', 'Id', 'Text', 'Date_Unix']
df.head(10)

,From_id,Id,Text,Date_Unix
0,-132,32494,"Картинка, ссылка или репост",1541422799
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555
2,-132,32492,"Картинка, ссылка или репост",1540743239
3,-132,32491,"Картинка, ссылка или репост",1540551399
4,-132,32488,"Картинка, ссылка или репост",1540302306
5,-132,32486,26-28 октября в Москве пройдет крупнейший в Ро...,1539863523
6,-132,32485,"Картинка, ссылка или репост",1539781575
7,-132,32484,"Картинка, ссылка или репост",1539718883
8,-132,32481,"Картинка, ссылка или репост",1539088885
9,-132,32480,Будущим магистрам! Новая программа Факультета ...,1539088745


С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях. Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат год-месяц-день-часы-минуты-секунды. Для этого нам понадобится модуль datetime.

In [48]:
from datetime import datetime

In [50]:
def date_norm(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:минуты:секунды
    date_norm, time_norm = str_d.split(' ') # разобьем результат на части, отделим дату от времени
    return date_norm, time_norm

Применим нашу функцию к элементам столбца date и создадим новый ‒ `date_norm`.

In [51]:
df['Date_Norm'] = df.Date_Unix.apply(date_norm)

In [52]:
df.head()

,From_id,Id,Text,Date_Unix,Date_Norm
0,-132,32494,"Картинка, ссылка или репост",1541422799,"(2018-11-05, 15:59:59)"
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,"(2018-11-01, 15:49:15)"
2,-132,32492,"Картинка, ссылка или репост",1540743239,"(2018-10-28, 19:13:59)"
3,-132,32491,"Картинка, ссылка или репост",1540551399,"(2018-10-26, 13:56:39)"
4,-132,32488,"Картинка, ссылка или репост",1540302306,"(2018-10-23, 16:45:06)"


Можно было, конечно, не разбивать на части дату и время, сохранять одной строкой. А можно написать функции, которые будут отделять дату от времени ‒ извлекать их из кортежа в date_norm.

In [53]:
def get_date(date):
    return date[0]

def get_time(date):
    return date[1]

In [54]:
df['Date'] = df.Date_Norm.apply(get_date)
df['Time'] = df.Date_Norm.apply(get_time)
df.head()

,From_id,Id,Text,Date_Unix,Date_Norm,Date,Time
0,-132,32494,"Картинка, ссылка или репост",1541422799,"(2018-11-05, 15:59:59)",2018-11-05,15:59:59
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,"(2018-11-01, 15:49:15)",2018-11-01,15:49:15
2,-132,32492,"Картинка, ссылка или репост",1540743239,"(2018-10-28, 19:13:59)",2018-10-28,19:13:59
3,-132,32491,"Картинка, ссылка или репост",1540551399,"(2018-10-26, 13:56:39)",2018-10-26,13:56:39
4,-132,32488,"Картинка, ссылка или репост",1540302306,"(2018-10-23, 16:45:06)",2018-10-23,16:45:06


Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://vk.com/dev/manuals).

## Координаты МКС

Тут можно посмотреть положение МКС http://open-notify.org/Open-Notify-API/ISS-Location-Now/

Ссылка для получения json: http://api.open-notify.org/iss-now.json

In [37]:
import json
d = requests.get('http://api.open-notify.org/iss-now.json').json()

In [40]:
json.loads(requests.get('http://api.open-notify.org/iss-now.json').text)

{'message': 'success',
 'timestamp': 1679138076,
 'iss_position': {'latitude': '-2.2270', 'longitude': '71.7875'}}

In [37]:
d['iss_position']['longitude']

'114.9936'

In [41]:
from datetime import datetime

In [44]:
datetime.fromtimestamp(1679138076).strftime("%Y-%m-%d %H:%M:%S")

'2023-03-18 14:14:36'

In [45]:
def get_date(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:минуты:секунды
    return str_d

In [31]:
get_date(1665229354)

'2022-10-08 14:42:34'

In [46]:
def get_iss_position():
    s = requests.get('http://api.open-notify.org/iss-now.json').json()
    return get_date(s['timestamp']), s['iss_position']['latitude'], s['iss_position']['longitude']
    

In [48]:
get_iss_position()

('2023-03-18 14:20:24', '15.2795', '84.5740')

В какой стране находятся координаты:
https://www.geonames.org/

Тут нужно зарегистрироваться

In [52]:
def get_country(lat, long):
    url = 'http://api.geonames.org/countryCodeJSON'
    parameters = {
        'lat':lat,
        'lng':long,
        'username': 'pileyan'
    }
    return requests.get(url, parameters).json()
# ['countryName']

In [53]:
date, latitude, longitude = get_iss_position()
country = get_country(latitude, longitude)

In [54]:
country

{'languages': 'en-IN,hi,bn,te,mr,ta,ur,gu,kn,ml,or,pa,as,bh,sat,ks,ne,sd,kok,doi,mni,sit,sa,fr,lus,inc',
 'distance': '0',
 'countryCode': 'IN',
 'countryName': 'India'}

In [55]:
def iss_monitor():
    date, latitude, longitude = get_iss_position()
    country = get_country(latitude, longitude)
    return {'date': date, 'latitude': latitude, 'longitude': longitude, 'country': country}

In [56]:
iss_monitor()

{'date': '2023-03-18 14:24:20',
 'latitude': '26.6684',
 'longitude': '94.5173',
 'country': {'languages': 'en-IN,hi,bn,te,mr,ta,ur,gu,kn,ml,or,pa,as,bh,sat,ks,ne,sd,kok,doi,mni,sit,sa,fr,lus,inc',
  'distance': '0',
  'countryCode': 'IN',
  'countryName': 'India'}}

In [57]:
import time

In [58]:
time.sleep(1)

In [59]:
lst = []
for i in range(75):
    lst.append(iss_monitor())
    time.sleep(5)

KeyboardInterrupt: 

In [61]:
lst

[{'date': '2023-03-18 14:24:28',
  'latitude': '27.0173',
  'longitude': '94.8644',
  'country': {'languages': 'en-IN,hi,bn,te,mr,ta,ur,gu,kn,ml,or,pa,as,bh,sat,ks,ne,sd,kok,doi,mni,sit,sa,fr,lus,inc',
   'distance': '0',
   'countryCode': 'IN',
   'countryName': 'India'}},
 {'date': '2023-03-18 14:24:33',
  'latitude': '27.2726',
  'longitude': '95.1205',
  'country': {'languages': 'en-IN,hi,bn,te,mr,ta,ur,gu,kn,ml,or,pa,as,bh,sat,ks,ne,sd,kok,doi,mni,sit,sa,fr,lus,inc',
   'distance': '0',
   'countryCode': 'IN',
   'countryName': 'India'}},
 {'date': '2023-03-18 14:24:39',
  'latitude': '27.5273',
  'longitude': '95.3778',
  'country': {'languages': 'en-IN,hi,bn,te,mr,ta,ur,gu,kn,ml,or,pa,as,bh,sat,ks,ne,sd,kok,doi,mni,sit,sa,fr,lus,inc',
   'distance': '0',
   'countryCode': 'IN',
   'countryName': 'India'}},
 {'date': '2023-03-18 14:24:44',
  'latitude': '27.7814',
  'longitude': '95.6365',
  'country': {'languages': 'en-IN,hi,bn,te,mr,ta,ur,gu,kn,ml,or,pa,as,bh,sat,ks,ne,sd,kok,d

In [69]:
import requests
from selenium import webdriver

driver = webdriver.Firefox()

In [70]:
driver.get('https://reverb.com/marketplace?query=paoletti%20%20nancy')

In [72]:
parsed = BeautifulSoup(driver.page_source)

In [80]:
import numpy as np

In [89]:
mean = np.mean([float(i.text[1:].replace(',','')) for i in parsed.find_all('span', {'class':'price-display'})])

In [88]:
USD_currency = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']['USD']['Value']

In [90]:
mean*USD_currency

404102.1790376471